In [1]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv

import os, requests

In [2]:
load_dotenv()

True

In [3]:
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool

In [4]:
def browse_web(search_term: str):
    """
    search_term: search term to search for information from the web
    returns the information retireved for the corresponding search term
    """
    serper = GoogleSerperAPIWrapper()
    return serper.run(search_term)

In [5]:
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_user = os.getenv("PUSHOVER_USER")
pushover_url = "https://api.pushover.net/1/messages.json"

def push(text: str):
    """Send a push notification to the user"""
    requests.post(pushover_url, data = {"token": pushover_token, "user": pushover_user, "message": text})

In [6]:
def get_admin_details(_:str):
    return {"name":"Ashok","email":"ashok@gmail.com"}

In [7]:
tool_browse = Tool(
    name="browse_internet",
    func=browse_web,
    description="This tool is used to search the web for a given search term"
)

tool_push = Tool(
    name="push_notification",
    func=push,
    description="This tool is used to send push notification"
)

tool_admin = Tool(
    name="get_admin_details",
    func=get_admin_details,
    description="This tool is to get the admin details (Email & personal details of Admin)"
)


tools = [tool_browse,tool_push,tool_admin]

In [16]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini",temperature=0).bind_tools(tools)


In [9]:
tool_admin.invoke("Dummy")

{'name': 'Ashok', 'email': 'ashok@gmail.com'}

In [10]:
from langchain_core.messages import HumanMessage
from langgraph.graph import MessagesState,StateGraph,END


In [22]:
def run_reasoning_engine(state:MessagesState) -> MessagesState:
    """
    Run the reasoning engine
    """

    system_content = """
        You are a smart assistant designed to use tools efficiently to answer user queries
    """
    response = llm.invoke([{"role":"system", "content":system_content},*state["messages"]])
    return {"messages":response}


In [23]:
from langgraph.prebuilt import ToolNode,tools_condition

def tools_condition_manual(state: MessagesState) -> str:
    if not state["messages"][-1].tools_call:
        return END
    return "reasoning_engine"

In [39]:
# Adding memory import 
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
config = {"configurable":{"thread_id":"1"}}

flow = StateGraph(MessagesState)

ACT = "reasoning_engine"
TOOL = "tools"

flow.add_node(ACT,run_reasoning_engine)
flow.add_node(TOOL,ToolNode(tools=tools))
flow.set_entry_point("reasoning_engine")

flow.add_conditional_edges(ACT,tools_condition,TOOL)

# flow.add_conditional_edges("reasoning_engine",tools_condition_manual,{
#     END:END,
#     "act":"reasoning_engine"
# })

flow.add_edge(TOOL,ACT)
compiled_graph = flow.compile(checkpointer=memory);


In [40]:
print(compiled_graph.get_graph().draw_ascii())


        +-----------+         
        | __start__ |         
        +-----------+         
               *              
               *              
               *              
     +------------------+     
     | reasoning_engine |     
     +------------------+     
          .         *         
        ..           **       
       .               *      
+---------+         +-------+ 
| __end__ |         | tools | 
+---------+         +-------+ 


In [41]:
def chat(user_input: str, history):
    """
    Maps user input to LLM
    """
    res = compiled_graph.invoke({"messages": {"role":"user","content":user_input}},config=config)
    return res["messages"][-1].content


In [42]:
import gradio as gr
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


In [ ]:
res = compiled_graph.invoke({"messages": [HumanMessage(content="What is the temperature in Tokyo? List it and then triple it")]},config=config)
print(res["messages"][-1].content)

You asked for the current temperature in Tokyo, and then you requested to list that temperature and triple it. I provided the temperature as 81°F and calculated the tripled value as 243°F.


In [20]:
res = compiled_graph.invoke({"messages": [HumanMessage(content="What is the temperature in Dubai in Celsius? ")]})
print(res["messages"][-1].content)

The current temperature in Dubai is 36°C.


In [21]:
res = compiled_graph.invoke({"messages": [HumanMessage(content="Give me info about the admin? ")]})
print(res["messages"][-1].content)

The admin's name is Ashok, and the email address is ashok@gmail.com.
